In [2]:
# Compare 2 Dataframes based on id and extract unique rows:
# unique rows are that have unique id, or duplicate id but difference in date is more than 2 days
# these are data generated from 2 diff systems

In [3]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [4]:
#importing and cleaning
file1 = pd.read_excel('file 1 - d.xlsx')
file2 = pd.read_excel('file 2 - db.xlsx')
df1 = file1#[['Compare 1','Compare 2']]
df2 = file2#[['Compare 1', 'Compare 2']]
df1['Compare 2'] = pd.to_datetime(df1['Compare 2'].astype(str), errors='coerce')
df2['Compare 2'] = pd.to_datetime(df2['Compare 2'])
df1['Compare 1'] = df1['Compare 1'].astype(int)
df2['Compare 1'] = df2['Compare 1'].astype(int)

In [5]:
# merging and extracting

# merge dfs
dfmrg = df1.merge(df2, on='Compare 1', how='outer',indicator=True)
dfmrg['value'] = abs(dfmrg['Compare 2_x'] - dfmrg['Compare 2_y']) > timedelta(days=2)

dfmrggrp = dfmrg.groupby('Compare 1', as_index=False)

#delete non unique
for name, group in dfmrggrp:
    group = group.groupby('Compare 2_x', as_index=False)
    for nama, groupa in group:
        if not (groupa['value'].sum() > 0):
            del groupa

#converting groupby object to df, it's not standard I just invented it now! :)
predf = dfmrggrp.fillna(np.nan)

# extracting uniq. rows that belong to first df
predf1 = predf[['Site', 'Compare 1', 'Admit Date', 'Disch Date', 'Compare 2_x',
       'Care Unit Disch Time', 'ICU Admit Time', 'ICU Discharge Time','_merge']][predf['_merge']!='right_only']

# extracting uniq. rows that belong to second df
predf2 = predf[['First ICU Admission Location In NS', 'Compare 1', 'Admission Date',
       'Hospital Discharge Date', 'Compare 2_y', 'Discharge Date']][predf['_merge']!='left_only']

#fdf = predf.append(dfmrg[dfmrg._merge != 'both']) >> this seems nothing :)

#exporting
predf1.to_excel('file1_edited.xlsx',merge_cells=False)
predf2.to_excel('file2_edited.xlsx',merge_cells=False)
#if premission denied that means there's a file with same name or things like that

PermissionError: [Errno 13] Permission denied: 'file1_edited.xlsx'